In [126]:
import json
import yaml
from openapi_spec_validator import validate
from openapi_spec_validator.readers import read_from_filename

In [127]:
json_file_path = "nfl/docs_raw.json"

with open(json_file_path, 'r') as j:
    contents = json.loads(j.read())

In [141]:
docs = [content for content in contents['Operations'] 
        if content['Deprecated'] == False and 'Legacy' not in content['Name']][:10]

In [142]:
#tags
tags = [{"name": tag['UniqueFunctionName'], "description": str(tag['Description'])} for tag in docs]

In [143]:
#paths
paths ={}
for path in docs:
    path_json = {}
    parameters =[]
    for parameter in path['UrlTemplateParameters']:
        responses = {"200":{"description":"okay"}}
        parameters.append({"name" : str(parameter["Name"]), "in":'path','description':parameter["Description"], 'required':parameter['Required'],'schema': {'type': parameter['Type']}})
    path_json[path["Method"].lower()] = {"tags":path['Tags'], 'summary': str(path['Description']), "operationId": str(path['OperationID']), 'parameters':parameters, "responses":responses}
    
    paths[path['Path']] = path_json
    

In [152]:
docs_dict = {"openapi": "3.0.1", "info": {"version": "v3","title": "Sports Data API" },
             "servers": [{"url": "https://api.sportsdata.io/v3/nfl"}],
             'tags':tags,
             'paths':paths,
               "components": {
                "securitySchemes": {
                  "ApiKeyAuth": {
                    "type": "apiKey",
                        "in": "header",
                            "name": "Ocp-Apim-Subscription-Key"
                          }
                        }
                      },
                  "security": [
            {
                  "ApiKeyAuth": []
                }
                  ]
             
     }

In [153]:
with open("nfl/docs_edit.json", "w") as outfile: 
    json.dump(docs_dict, outfile)
spec_dict, base_uri = read_from_filename('nfl/docs_edit.json')

# If no exception is raised by validate(), the spec is valid.
validate(spec_dict)

In [147]:
docs_dict

{'openapi': '3.0.1',
 'info': {'version': 'v3', 'title': 'Sports Data API'},
 'servers': [{'url': 'https://api.sportsdata.io/v3/nfl'}],
 'tags': [{'name': 'PushAuthenticationGameState',
   'description': 'Returns the push token for the game state channel'},
  {'name': 'AreGamesInProgress',
   'description': 'Returns true if there is at least one game being played at the time of the request or false if there are none.'},
  {'name': 'ByeWeeks',
   'description': 'Get bye weeks for the teams during a specified NFL season.'},
  {'name': 'SeasonCurrent',
   'description': 'Year of the current NFL season. This value changes at the start of the new NFL league year. The earliest season for Fantasy data is 2001. The earliest season for Team data is 1985. The earliest season for Fantasy data is 2001. The earliest season for Team data is 1985.'},
  {'name': 'SeasonLastCompleted',
   'description': 'Year of the most recently completed season. this value changes immediately after the Super Bowl. Th